# Implementing Boundary Conditions: From Linear to Nonlinear in FEM

In [9]:
using Ferrite
using SparseArrays
using Plots
using LinearAlgebra

In [10]:
# deliberately use include to include code 
# see e.g. https://docs.julialang.org/en/v1/manual/code-loading/ 
include("flemish-fish.jl")

In [11]:
?fdmmesh

search: fdmmesh



Generates a one-dimensional uniform mesh between point 0 and 1

Input N(1) is number of elements. Output is the one-dimensional mesh. 

[`mesh(::NTuple{1,Int})`](@ref)

---

Generates a two-dimensional mesh between the points (0,0) and (1,1) 

Input N(1) and N(2) are the number of elements in x and y direction. Output is the two-dimensional mesh. 

[`mesh(::NTuple{2,Int})`](@ref)

---

Generates a two-dimensional mesh between the points a and b 

Input N(1) and N(2) are the number of elements in x and y direction. Output is the two-dimensional mesh. 

[`mesh(::NTuple{2,Int},::Point2D,::Point2D)`](@ref)


## Introduction 

This notebook outlines a structured approach to understanding and implementing boundary conditions (BCs) in numerical simulations. O

1.  **Hand-Coding Linear Dirichlet Boundary Conditions:** We will start by examining how existing libraries typically apply linear Dirichlet BCs, where a specific degree of freedom $\mathbf{u}_i$ is set to a fixed value $C$ (i.e., $\mathbf{u}_i = C$). Our focus will be on understanding how this condition translates into direct modifications of the global stiffness matrix $\mathbf{K}$ and the right-hand side vector $\mathbf{f}$. We will then proceed to "hand-code" this modification ourselves, learning how specific rows and columns of these matrices are altered to impose the constraint.

2.  **Implementing Affine Constraints:** Building upon the understanding of linear Dirichlet conditions, we will extend our capabilities to more general **affine constraints**. These are linear relationships involving multiple degrees of freedom, expressed as $\sum_{j} a_j \mathbf{u}_j = C$. A common example is enforcing a mean value constraint on a field variable over a certain region. This step will further develop our intuition for how linear dependencies are embedded within the algebraic system.

3.  **Extending to Nonlinear Boundary Conditions via Newton's Method:** Finally, we will apply the insights gained from handling linear constraints to the realm of nonlinear boundary conditions. For a nonlinear problem, the governing equations, including boundary conditions, form a system of nonlinear algebraic equations, $\mathbf{F}(\mathbf{u}) = \mathbf{0}$. We will solve this system using the Newton-Raphson method, which relies on solving a linearized system at each iteration: $J(\mathbf{u}^k) \mathbf{\delta u} = -\mathbf{F}(\mathbf{u}^k)$. Here, $J(\mathbf{u}^k)$ is the Jacobian matrix. Our key focus will be on how the nonlinear boundary condition (e.g., $[\rho_g(0)]^2 = 2$ from our previous work) is integrated into the residual function $\mathbf{F}(\mathbf{u})$ and, crucially, how its linearization appears within the Jacobian $J(\mathbf{u})$. This consolidates the understanding that even a nonlinear boundary condition translates to a linear contribution to the Jacobian system solved at each Newton iteration.

## Section 1 : Hand-Coded Linear Dirichlet Boundary Condition 

###  How linear Dirichlet boundary conditions are implemented using Ferrite ?

In this section, we will revisit our existing code from **section 4.1** of the "HydrogenProject" notebook : https://github.com/AnouchkaDESMETTRE/HydrogenProject/blob/main/HydrogenProject.ipynb, which addresses the steady-state gas density absorption equation:

$$D \frac{d^2 \rho_g}{dz^2} + u_z \frac{d \rho_g}{dz} + \dot{m} = 0$$

Our primary focus here will be to deeply understand how linear Dirichlet boundary conditions are implemented using Ferrite's `apply!(K,f,ch)` for FEM). Specifically, we will investigate how the system matrix $K$ and the right-hand side vector $f$ are directly modified. The goal is to explicitly "hand-code" the implementation of this Dirichlet boundary condition. This means we will bypass the high-level `applybc!` function for the Dirichlet constraint and instead directly manipulate the relevant row(s) of the discretized matrix and vector to enforce the condition $\rho_g(z_{boundary}) = C$. This exercise will provide a granular understanding of the algebraic changes required to impose such constraints.

In [36]:
# Problem parameters
L = 0.8
D = 1.0 
uz = 0.01
m_dot = 0.5
N = 5

interpolation = Lagrange{RefLine, 1}()
qr = QuadratureRule{RefLine}(2)
cell_values = CellValues(qr, interpolation)

# Définition précise du domaine
left = Vec((0.,))
right = Vec((L,))
grid = generate_grid(Ferrite.Line, (N,), left, right)

dh = DofHandler(grid)
add!(dh, :rho_g, interpolation)
close!(dh)

K = allocate_matrix(dh)
f = zeros(ndofs(dh))

function assemble!(K, f, cv, dh, D, uz, m_dot)
    assembler = start_assemble(K, f)
    for cell in CellIterator(dh)
        Ferrite.reinit!(cv, cell)
        n_basefuncs = getnbasefunctions(cv)
        Ke = zeros(n_basefuncs, n_basefuncs)
        fe = zeros(n_basefuncs)
        for q in 1:getnquadpoints(cv)
            dΩ = getdetJdV(cv, q)
            for i in 1:n_basefuncs
                ϕ = shape_value(cv, q, i)
                ∇ϕ = shape_gradient(cv, q, i)[1]
                for j in 1:n_basefuncs
                    ∇ϕ_j = shape_gradient(cv, q, j)[1]
                    Ke[i, j] += (D * ∇ϕ * ∇ϕ_j - uz * ϕ * ∇ϕ_j) * dΩ
                end
                fe[i] += m_dot * ϕ * dΩ
            end
        end
        Ferrite.assemble!(assembler, celldofs(cell), Ke, fe)
    end
end

assemble!(K, f, cell_values, dh, D, uz, m_dot)

ch = ConstraintHandler(dh)
dbc1 = Dirichlet(:rho_g, [1], (x, t) -> 1.0) # Appliquer au nœud 1
dbc2 = Dirichlet(:rho_g, [Ferrite.getnnodes(grid)], (x, t) -> 0.0) # Appliquer au dernier nœud
add!(ch, dbc1)
add!(ch, dbc2)
close!(ch)

# --- Affichage AVANT apply! ---
println("Matrice K AVANT apply! (N=$N):")
display(Array(K)) # Convertir en Dense Array pour une meilleure visualisation des zéros
println("\n")
println("Vecteur f AVANT apply! (N=$N):")
display(f)
println("---------------------------------------------------------")

apply!(K, f, ch)

rho_g = K \ f

rho_g_computed_at_nodes = evaluate_at_grid_nodes(dh, rho_g, :rho_g);

# Génération des coordonnées z à partir des nœuds du maillage
z_coords = [grid.nodes[node].x[1] for node in 1:length(grid.nodes)];

## Initialiser le plot
#p_solution = plot(title="FEM Solution (N=$N)",
#    xlabel="z (m)", ylabel="ρg (kg/m³)", lw=2, legend=false) # legend=false car une seule courbe

## Plot de la solution FEM
#plot!(p_solution, z_coords, rho_g_computed_at_nodes, label="FEM Solution", lw=2, markershape=:circle, markersize=3)

## Afficher le plot
#display(p_solution)

Matrice K AVANT apply! (N=5):


6×6 Matrix{Float64}:
  6.255  -6.255   0.0     0.0     0.0     0.0
 -6.245  12.5    -6.255   0.0     0.0     0.0
  0.0    -6.245  12.5    -6.255   0.0     0.0
  0.0     0.0    -6.245  12.5    -6.255   0.0
  0.0     0.0     0.0    -6.245  12.5    -6.255
  0.0     0.0     0.0     0.0    -6.245   6.245



Vecteur f AVANT apply! (N=5):


6-element Vector{Float64}:
 0.039999999999999994
 0.07999999999999999
 0.07999999999999999
 0.07999999999999999
 0.08
 0.04

---------------------------------------------------------


**Stifness Matrix $K$ BEFORE `apply!`**

The values of the coefficients directly reflect the assembly of the element matrices for diffusion ($D/h$) and convection ($u\_z/2$).

* $K[1,1] = 6.255$ corresponds to the sum of diffusion and convection contributions for the first node: $\left(\frac{D}{h} + \frac{u_z}{2}\right) = (1.0/0.16 + 0.01/2) = 6.25 + 0.005 = 6.255$.

* Similarly, $K[1,2] = -6.255$ corresponds to $\left(-\frac{D}{h} - \frac{u_z}{2}\right) = (-1.0/0.16 - 0.01/2) = -6.25 - 0.005 = -6.255$.

* Diagonal terms for internal nodes, such as $K[2,2] = 12.5$, are approximately $\frac{2D}{h}$, as these nodes receive contributions from two adjacent elements ($2 \times 6.25 = 12.5$).

**Force Vector $f$ BEFORE `apply!`**

The values of the components are calculated by the integral $\int \dot{m} \phi_i \, d\Omega$.

* For the nodes at the domain boundaries (nodes 1 and 6), the contribution is $\dot{m} \cdot \frac{h}{2} = 0.5 \cdot (0.8/5) / 2 = 0.04$. This is observed for $f[6]$ and is very close for $f[1]$.

* For internal nodes (nodes 2, 3, 4, 5), the contribution is the sum of contributions from two adjacent elements, i.e., $\dot{m} \cdot h = 0.5 \cdot 0.16 = 0.08$. This is verified for $f[2]$ through $f[5]$.

In [37]:
# --- Affichage APRÈS apply! ---
println("Matrice K APRÈS apply! (N=$N):")
display(Array(K)) # Convertir en Dense Array
println("\n")
println("Vecteur f APRÈS apply! (N=$N):")
display(f)
println("\n")
# -----------------------------

Matrice K APRÈS apply! (N=5):


6×6 Matrix{Float64}:
 10.4167   0.0     0.0     0.0     0.0     0.0
  0.0     12.5    -6.255   0.0     0.0     0.0
  0.0     -6.245  12.5    -6.255   0.0     0.0
  0.0      0.0    -6.245  12.5    -6.255   0.0
  0.0      0.0     0.0    -6.245  12.5     0.0
  0.0      0.0     0.0     0.0     0.0    10.4167



Vecteur f APRÈS apply! (N=5):


6-element Vector{Float64}:
 10.416666666666664
  6.324999999999999
  0.07999999999999999
  0.07999999999999999
  0.08
  0.0

The applied Dirichlet boundary conditions are $\\rho\_g(z=0)=1.0$ (node 1) and $\\rho\_g(z=L)=0.0$ (node 6).

**Stifness Matrix $K$ AFTER `apply!`**

* The **first row** of $K$ (corresponding to node 1) has been significantly modified. $K[1,1]$ changed from $6.255$ to $10.4167$, and all other terms in this row ($K[1,2]$ to $K[1,6]$) became $0.0$.
  
* Similarly, the **last row** (corresponding to node 6) saw $K[6,6]$ change from $6.245$ to $10.4167$, with other terms in the row ($K[6,1]$ to $K[6,5]$) set to $0.0$.

* The **columns** corresponding to the constrained nodes (column 1 and column 6) have also been modified, with terms like $K[2,1]$ and $K[5,6]$ being set to $0.0$.

**Force Vector $f$ AFTER `apply!`**

* $f[1]$ changed from $0.0399...$ (source term) to $10.4166...$. This new value is the imposed boundary condition value at node 1 (which is $1.0$) multiplied by the factor applied to the diagonal of $K$ (here $10.4167$). Indeed, the equation for node 1 now effectively becomes $10.4167 \cdot \rho_{g,1} = 10.41666...$, which implies $\\rho\_{g,1} = 1.0$.

`apply!` implements strong Dirichlet boundary conditions using the *"large value on the diagonal method"*. This is a practical implementation of the "penalty method (see [1]) : 
For each constrained degree of freedom $i$ with imposed value $C$:
1.  Set $K[i,i]$ to a large factor (e.g., $10.4167$) and $K[i,j]$ to $0.0$ for $j \neq i$.
2.  Set $f[i]$ to (large factor) $\cdot C$.
3.  Adjust other $f[j]$ terms by transferring contributions from the constrained node ($f[j] = f[j] - K_{\text{original}}[j,i] \cdot C$).


* $f[6]$ changed from $0.04$ (source term) to $0.0$. Similarly, this is the BC value at node 6 (which is $0.0$) multiplied by the diagonal factor.

* Impact on Internal Nodes: The internal nodes ($f[3]$, $f[4]$, $f[5]$) retain their initial values from the source term, as their equations are not directly affected by this "row/column modification" method for the boundary nodes.

* However, $f[2]$ is affected by the treatment of the boundary condition at node 1. This is due to the "transfer" of contributions from the constrained node to the right-hand side of the equations for adjacent unconstrained nodes. 

Before applying the Dirichlet boundary condition at node 1, the second equation in our linear system $\mathbf{K}\mathbf{u} = \mathbf{f}$ (corresponding to node 2) would look like this:

$$K[2,1]\rho_{g,1} + K[2,2]\rho_{g,2} + K[2,3]\rho_{g,3} = f_{\text{original}}[2]$$

where $f_{\text{original}}[2]$ is the value on the right-hand side for node 2, arising solely from the integration of the source term $\dot{m}$.

When we impose a Dirichlet boundary condition $\rho_{g,1} = C_1$ (in our case, $C_1 = 1.0$), the value of $\rho_{g,1}$ is now known. The strategy used by the `apply!` function (and common in Finite Element Method implementations) is to "remove" the terms whose values are already known from the left-hand side of the equations (see [2]). The term $K[2,1]\rho_{g,1}$ is a contribution whose value is known (since $K[2,1]$ is a matrix coefficient and $\rho_{g,1}$ is our imposed value $C_1$). This known contribution is then moved from the left-hand side to the right-hand side of the equation : 

$$K[2,2]\rho_{g,2} + K[2,3]\rho_{g,3} = f_{\text{original}}[2] - K_{\text{original}}[2,1]\rho_{g,1}$$

Thus, the new value of $f[2]$ (let's call it $f_{\text{new}}[2]$) is calculated as follows:

$$f_{\text{new}}[2] = f_{\text{original}}[2] - K_{\text{original}}[2,1] \cdot C_1$$

Let's verify this with the values we obtained in our output:
* $f_{\text{original}}[2] = 0.07999999999999999 \approx 0.08$
* $K_{\text{original}}[2,1] = -6.245$
* $C_1 = 1.0$ (the value of $\rho_g$ at node 1 imposed by the BC)

Substituting these values:
$$f_{\text{new}}[2] = 0.08 - (-6.245) \cdot 1.0$$
$$f_{\text{new}}[2] = 0.08 + 6.245 = 6.325$$

And $f[2]$ after `apply!` is $6.324999999999999$, which is almost exactly $6.325$.



[1] Dr.Y.DILIPKUMAR (2025). Finite Element Analysis [https://mrcet.com/downloads/digital_notes/ME/IV%20year/Finite%20Element%20Analysis.pdf](https://mrcet.com/downloads/digital_notes/ME/IV%20year/Finite%20Element%20Analysis.pdf)]

[2] : Pr. S. Deparis. (2004). Numerical Analysis of Axisymmetric Flows and Methods for Fluid-Structure Interaction Arising in Blood Flow Simulation (Thèse No 2965). École Polytechnique Fédérale de Lausanne (EPFL)

### Manual Implementation of Linear Dirichlet Boundary Conditions

The observation of `apply!`'s effect on $K$ and $f$ reveals the commonly used strategy for implementing linear Dirichlet boundary conditions (where a value is imposed for a Degree of Freedom). To implement this "by hand" in our code, we must follow these steps for each DoF $i$ where a value $C$ is imposed ($u_i = C$):

1.  **Modify Row $i$ of the Stiffness Matrix $K$:**

      * Set the diagonal value $K[i,i]$ to a non-zero value. This is a "large value" (as observed with Ferrite, e.g., $10.4167$) to ensure numerical stability and diagonal dominance, especially when dealing with floating-point arithmetic.
      
      * Set all other elements in row $i$ (i.e., $K[i,j]$ for $j \neq i$) to $0.0$.

2.  **Modify Column $i$ of the Stiffness Matrix $K$:**

      * For each row $j$ where $j \neq i$, set the off-diagonal element $K[j,i]$ to $0.0$. This ensures that the equation for node $j$ does not explicitly depend on the unknown value of $u_i$ (which is already known and imposed).

3.  **Modify the Force Vector $f$:**

      * Set the element $f[i]$ to the imposed value $C$. If a "large value" factor (like $10.4167$) was used for $K[i,i]$, then $f[i]$ should be set to $\text{factor} \cdot C$. This ensures that the $i$-th equation becomes $\text{factor} \cdot u_i = \text{factor} \cdot C$, directly forcing $u_i= C$.
      
      * If we performed step 2 (setting column $i$ to zero), we must transfer the contribution of the known boundary condition value $C$ to the right-hand side (vector $f$) of the equations for other nodes. For each row $j$ where $j \neq i$:
        $$f[j]_{\text{new}} = f[j]_{\text{original}} - K_{\text{original}}[j,i] \cdot C$$
        where $K\_{\text{original}}[j,i]$ is the value of the matrix element before setting $K[j,i]$ to zero. This precisely accounts for the known value of $u_i$ in the equations of connected nodes.

These steps ensure that the equation for the constrained degree of freedom becomes trivially satisfied by the imposed value, while its influence on other equations is correctly accounted for, leading to a well-posed system for the remaining unknown degrees of freedom.

# Section 2: Affine Constraints as Boundary Conditions 

# Section 3: Extension to the Non-Linear Case 